# Assignment 1

In this assignment, you will investigate the precision issues in computing the gradient. You will also implement a simple linear regression model using the custom autograd engine.

## Task 1: Precision issues

In [1]:
def f(x):
    return 3 * x ** 2 - 4 * x + 5

def gradient(f, x, h=0.0001):
    return (f(x + h) - f(x)) / h

gradient(f, 2)

8.000300000023941

In [ ]:
# What happens if we keep decreasing h?
gradient(f, 2, h=0.0000000000000001)

################################################################################
# TODO:                                                                        #
# Why is the gradient 0?                                                       #
# If you don't know, google it!                                                #
################################################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
# ANSWER: 
# 부동 소수점 방식은 고정 소수점 방식보다 훨씬 더 많은 범위까지 표현할 수 있지만, 항상 오차가 존재한다는 단점을 가지고 있다.
# ±(1.가수부)×2^지수부-127 부동소수점의 오차를 알고 싶을 때 사용할 수 있는 방식

num = 0.1
for i in range(1000):
    num += 0.1
print(num)

#1000이 나와야 하지만 부동소수점 오차로 인해서 차이가 발생생

# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

100.09999999999859


## Task 2: Linear Regression

Let's review the training loop of a simple linear regression model.

### Importing Libraries

In [10]:
import math
import random

In [11]:
class Tensor:
    def __init__(self, data, _children=(), _operation=''):
        self.data = data
        self._prev = set(_children)
        self.gradient = 0
        self._backward = lambda: None

    def __repr__(self):
        return f"tensor=({self.data})"

    def __add__(self, other):  # self + other
        output = Tensor(self.data + other.data, (self, other), '+')
        def _backward():
            self.gradient = 1 * output.gradient
            other.gradient = 1 * output.gradient
        output._backward = _backward
        return output

    def __mul__(self, other):  # self * other
        output = Tensor(self.data * other.data, (self, other), '*')
        def _backward():
            self.gradient = other.data * output.gradient
            other.gradient = self.data * output.gradient
        output._backward = _backward
        return output

    def tanh(self):  # tanh(self)
        output = Tensor(math.tanh(self.data), (self,), 'tanh')
        def _backward():
            self.gradient = (1.0 - math.tanh(self.data) ** 2) * output.gradient
        output._backward = _backward
        return output

    def __pow__(self, power):  # self ** power
        assert isinstance(power, (int, float)), "Power must be an int or a float"
        output = Tensor(self.data ** power, (self,), f'**{power}')
        def _backward():
            self.gradient = power * (self.data ** (power - 1)) * output.gradient
        output._backward = _backward
        return output

    def backward(self):
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)
        self.gradient = 1
        for node in reversed(topo):
            node._backward()

    def __neg__(self): # -self
        return self * Tensor(-1.0)

    def __sub__(self, other): # self - other
        return self + (-other)

### Data

**GOAL: Find the best line that fits the following data.**

![Data](../../assets/linear.png)

(Image credit: MIT 18.06)

(1, 1), (2, 2), (3, 2)

In [12]:
# Input, Target data
input = [Tensor(1), Tensor(2), Tensor(3)]
target = [Tensor(1), Tensor(2), Tensor(2)]

### Model

In [13]:
# Linear regression model
class Linear:
    def __init__(self):
        self.a = Tensor(random.uniform(-1, 1))
        self.b = Tensor(random.uniform(-1, 1))

    def __call__(self, x):
        y = self.a * x + self.b
        return y

    def parameters(self):
        return self.a, self.b

In [14]:
# Initialize the model
model = Linear()

# Example forward pass
print(f"Output: {model(input[0])}")

Output: tensor=(0.9012000861451694)


### Training

Implement the training loop for the linear regression model.

Choose an appropriate learning rate.

In [74]:
lr = 2e-4  # learning rate

# Training loop
for step in range(10):
    total_loss = Tensor(0)
    
    # Forward pass
    for x, y in zip(input, target):
        ################################################################################
        # TODO:                                                                        #
        # Implement the forward pass.                                                  #
        ################################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        y_pred = model(x)

        loss = (y_pred - y) ** 2
        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        total_loss += loss

    # Backward pass
    ################################################################################
    # TODO:                                                                        #
    # Implement the backward pass.                                                 #
    ################################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    total_loss.backward()
    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    # Update weights
    ################################################################################
    # TODO:                                                                        #
    # Update the weights of the model using the gradients.                         #
    ################################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    for param in model.parameters():
        param.data -= lr * param.gradient  # Gradient Descent
    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
            
    print(f"Step: {step}, Loss: {total_loss.data}")

Step: 0, Loss: 0.4191251902580625
Step: 1, Loss: 0.41936403599711
Step: 2, Loss: 0.4196084944907211
Step: 3, Loss: 0.4193438316997179
Step: 4, Loss: 0.4195886282463308
Step: 5, Loss: 0.4198389573989507
Step: 6, Loss: 0.4200947450370044
Step: 7, Loss: 0.41981630460262404
Step: 8, Loss: 0.4200723555206752
Step: 9, Loss: 0.420333787224002


In [75]:
# Print the final weights of the model
print(f"y = {model.a.data}*x + {model.b.data}")

y = 0.8148001965173421*x + -0.09923673798875766


## Extra Credit

Linear regression is the simplest form of neural networks. It actually does not require gradient descent to solve for the weights.

**Find a way to get the weights of the linear regression model without using gradient descent.**

In [ ]:
################################################################################
# TODO:                                                                        #
# y = ax + b                                                                   #
# x, y = (1, 1), (2, 2), (3, 2)                                                #
# Find the values of a and b without using gradient descent.                   #
################################################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
# a = ?? -> 0.5
# b = ?? -> 0.666666....
# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****